In [789]:
#import dependencies
import pandas as pd
import numpy

In [790]:
#create file paths
school_file = "raw_data/schools_complete.csv"
students_file = "raw_data/students_complete.csv"

In [791]:
#read data into a pandas dataset
schools_df = pd.read_csv(school_file)
students_df = pd.read_csv(students_file)

In [792]:
#start district summary by finding the total number of schools
total_schools = schools_df["name"].count()
total_schools

15

In [793]:
#district summary--find the total number of students in the district
total_students = students_df["name"].count()
total_students

39170

In [794]:
#district analysis -- find total budget
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [795]:
#district analysis -- calculate the average scores for reading and math 
district_average_reading = students_df["reading_score"].mean()
print(district_average_reading)
district_average_math = students_df["math_score"].mean()
print(district_average_math)

81.87784018381414
78.98537145774827


In [796]:
#calculate the total passing in math
pass_count_math = students_df[students_df["math_score"] > 69].count()
pass_count_math = pass_count_math["math_score"]
pass_count_math

29370

In [797]:
#calculate the total passing in reading
pass_count_reading = students_df[students_df["reading_score"] > 69].count()
pass_count_reading = pass_count_reading["reading_score"]
pass_count_reading

33610

In [798]:
#districct analysis -- calculate the percentage passing in reading and math
percent_passing_reading = pass_count_reading / total_students
percent_passing_math = pass_count_math / total_students
print ('{:.2%}'.format(percent_passing_math))
print('{:.2%}'.format(percent_passing_reading))

74.98%
85.81%


In [799]:
#calculate the overall passing rate
overall_pass = (percent_passing_math + percent_passing_reading) / 2
'{:.2%}'.format(overall_pass)

'80.39%'

In [800]:
#create a summary table of the district's key metrics
district_summary_table = pd.DataFrame(
    {
        "Total Schools": '{:,.0f}'.format(total_schools),
        "Total Students": '{:,.0f}'.format(total_students),
        "Total Budget": '${:,.2f}'.format(total_budget),                              
        "Average Math Score": '{:,.2f}'.format(district_average_math),
        "Average Reading Score": '{:,.2f}'.format(district_average_reading),
        "% Passing Math":'{:.2%}'.format(percent_passing_math),
        "% Passing Reading":'{:.2%}'.format(percent_passing_reading),
        "Overall Passing Rate": '{:.2%}'.format(overall_pass)
    }, index=[0])
district_summary_table

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,74.98%,85.81%,78.99,81.88,80.39%,"$24,649,428.00",15,"39,170"


In [801]:
#District Summary complete
#Begin analysis of the individual schools
#build upon the schools_df by adding the summary columns from the students_df

In [802]:
#review the beginning of the dataset to familiarize
students_df = students_df.rename(columns={"name": "Student Name", "school": "School Name"})
students_df.head()

,Student ID,Student Name,gender,grade,School Name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [803]:
#review the beginning of the dataset to familiarize
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [804]:
#rename the "size" in the table to total students
schools_summary_table = schools_df.rename(columns={"name": "School Name", 'type': "School Type", 'size': 'Total Students', 'budget': "Total Budget"})

In [805]:
#calculate the budge per student and append a column to the summary table
schools_summary_table["Per Student Budget"] = schools_summary_table["Total Budget"] / schools_summary_table["Total Students"]

In [806]:
#calculate the average scores for each school
#group the student_df by school and check the .mean() method
#test--create new dataframe from groupby object
df = pd.DataFrame(students_df.groupby("School Name").mean())
df = df.reset_index()

In [807]:
schools_summary_table = pd.merge(schools_summary_table, df, on="School Name")
schools_summary_table = schools_summary_table.rename(columns={"reading_score": "Average Reading Score", "math_score": "Average Math Score"})

In [808]:
#Calculate the number of students who passed reading (for calculating the percent pass later)
df2 = students_df[["School Name", "reading_score"]]
df2 = df2.loc[df2["reading_score"] > 69]
df2 = df2.groupby("School Name").count()
df2 = df2.rename(columns={"reading_score": "# of Pass Reading"})
df2 = df2.reset_index()

In [809]:
#Calculate the number of students who passed math (for calculating the percent pass later)
df3 = students_df[["School Name", "math_score"]]
df3 = df3.loc[df3["math_score"] > 69]
df3 = df3.groupby("School Name").count()
df3 = df3.rename(columns={"math_score": "# of Pass Math"})
df3 = df3.reset_index()

In [810]:
#merge the #'s passed dataframes with the summary df in order to perform calculations
schools_summary_table = pd.merge(schools_summary_table, df2, on="School Name")
schools_summary_table = pd.merge(schools_summary_table, df3, on="School Name")

In [811]:
#Calculate the passing % for math and reading using the columns in the dataframe and add them as new columns
schools_summary_table["% Passing Reading"] = schools_summary_table["# of Pass Reading"] / schools_summary_table["Total Students"]
schools_summary_table["% Passing Math"] = schools_summary_table["# of Pass Math"] / schools_summary_table["Total Students"]
schools_summary_table["Overall Passing Rate"] = (schools_summary_table["% Passing Reading"] + schools_summary_table["% Passing Math"]) / 2

In [812]:
#drop the unnecessary columns from the summary dataframe
schools_summary_table = schools_summary_table.drop(["School ID", "# of Pass Reading", "# of Pass Math", "Student ID"], axis=1)
schools_summary_table.head()

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,0.958546,0.938671,0.948609
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.808630,0.667530,0.738080
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.952657


In [813]:
schools_summary_table.head()

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,0.958546,0.938671,0.948609
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.808630,0.667530,0.738080
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.952657


In [814]:
# Summary Table for Individual Schools complete
# Begin creating table for the top 5 performing schools based on overall pass rate

In [815]:
# sort the summary dataframe in order to create a df in which the top five schools are on the top
top5 = schools_summary_table.sort_values("Overall Passing Rate", ascending=False).reset_index(drop=True)
top5 = top5.iloc[0:5].set_index("School Name")
top5

,School Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,0.970398,0.941335,0.955867
Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,0.973089,0.932722,0.952905
Pena High School,Charter,962,585858,609.0,84.044699,83.839917,0.959459,0.945946,0.952703
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.952657
Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,0.965396,0.938677,0.952037


In [816]:
# sort the summary dataframe in order to create a df in which the bottom five schools are on the top
bottom5 = schools_summary_table.sort_values("Overall Passing Rate", ascending=True).reset_index(drop=True)
bottom5 = bottom5.iloc[0:5].set_index("School Name")
bottom5

,School Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,0.802201,0.663666,0.732933
Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,0.812224,0.660576,0.736400
Ford High School,District,2739,1763916,644.0,80.746258,77.102592,0.792990,0.683096,0.738043


In [817]:
#top and bottom 5 tables complete
#create table that sorts math scores by school and grade level

In [818]:
#take a peek at the student df again
students_df.head()

,Student ID,Student Name,gender,grade,School Name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [819]:
#Create table for math scores by grade
math_scores_by_grade = students_df.groupby(["School Name", "grade"])["math_score"].mean()
math_scores_by_grade = pd.DataFrame(math_scores_by_grade).unstack()
math_scores_by_grade = math_scores_by_grade[math_scores_by_grade.columns[::-1]]
math_scores_by_grade

math_score                                 
grade                        9th       12th       11th       10th
School Name                                                      
Bailey High School     77.083676  76.492218  77.515588  76.996772
Cabrera High School    83.094697  83.277487  82.765560  83.154506
Figueroa High School   76.403037  77.151369  76.884344  76.539974
Ford High School       77.361345  76.179963  76.918058  77.672316
Griffin High School    82.044010  83.356164  83.842105  84.229064
Hernandez High School  77.438495  77.186567  77.136029  77.337408
Holden High School     83.787402  82.855422  85.000000  83.429825
Huang High School      77.027251  77.225641  76.446602  75.908735
Johnson High School    77.187857  76.863248  77.491653  76.691117
Pena High School       83.625455  84.121547  84.328125  83.372000
Rodriguez High School  76.859966  77.690748  76.395626  76.612500
Shelton High School    83.420755  83.778976  83.383495  82.917411
Thomas High School     83.590022  83.497041  83.498795  83.087886
Wilson High School     83.085578  83.035794  83.195326  83.724422
Wright High School     83.264706  83.644986  83.836782  84.010288

In [820]:
#create table for reading scores by grade
reading_scores_by_grade = students_df.groupby(["School Name", "grade"])["reading_score"].mean()
reading_scores_by_grade = pd.DataFrame(reading_scores_by_grade).unstack()
reading_scores_by_grade = reading_scores_by_grade[reading_scores_by_grade.columns[::-1]]
reading_scores_by_grade

reading_score                                 
grade                           9th       12th       11th       10th
School Name                                                         
Bailey High School        81.303155  80.912451  80.945643  80.907183
Cabrera High School       83.676136  84.287958  83.788382  84.253219
Figueroa High School      81.198598  81.384863  80.640339  81.408912
Ford High School          80.632653  80.662338  80.403642  81.262712
Griffin High School       83.369193  84.013699  84.288089  83.706897
Hernandez High School     80.866860  80.857143  81.396140  80.660147
Holden High School        83.677165  84.698795  83.815534  83.324561
Huang High School         81.290284  80.305983  81.417476  81.512386
Johnson High School       81.260714  81.227564  80.616027  80.773431
Pena High School          83.807273  84.591160  84.335938  83.612000
Rodriguez High School     80.993127  80.376426  80.864811  80.629808
Shelton High School       84.122642  82.781671  84.373786  83.441964
Thomas High School        83.728850  83.831361  83.585542  84.254157
Wilson High School        83.939778  84.317673  83.764608  84.021452
Wright High School        83.833333  84.073171  84.156322  83.812757

In [821]:
schools_summary_table.head()

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,0.958546,0.938671,0.948609
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.808630,0.667530,0.738080
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.971390,0.933924,0.952657


In [829]:
#create table for scores by school spending
scores_by_spending = schools_summary_table
#create bins and bin lables
bins = [0, 600, 620, 640, 655]
spending_ranges = ["Under $600", "$600-620", "$620-640", "Over $640"]
#create new column for the binned data
scores_by_spending["Spending Ranges (Per Student)"] = pd.cut(scores_by_spending["Per Student Budget"], bins, labels=spending_ranges)
#group the df by the binned column and create new df for it
scores_by_spending = scores_by_spending.groupby("Spending Ranges (Per Student)").mean()
scores_by_spending = scores_by_spending
#drop the irrelevant columns from the df

,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
Spending Ranges (Per Student),,,,,,,,
Under $600,1625.8,951003.40,584.80,83.892196,83.436210,0.964596,0.935415,0.950006
$600-620,962.0,585858.00,609.00,84.044699,83.839917,0.959459,0.945946,0.952703
$620-640,3005.4,1903466.40,633.40,82.120471,79.474551,0.874681,0.771399,0.823040
Over $640,3763.0,2447805.25,650.25,80.957446,77.023555,0.806752,0.667010,0.736881


In [778]:
#apply formatting to the columns in the school summary df
format_percent = "{0:.2f}%".format
format_round2 =  '{:,.2f}'.format
format_round0 = '{:,.0f}'.format
format_currency = '${:,.2f}'.format
schools_summary_table[['% Passing Reading','% Passing Math', 'Overall Passing Rate']] = schools_summary_table[['% Passing Reading','% Passing Math', 'Overall Passing Rate']] * 100
schools_summary_table[['Total Budget','Per Student Budget']] = schools_summary_table[['Total Budget','Per Student Budget']].applymap(format_currency)
schools_summary_table[['Average Reading Score','Average Math Score']] = schools_summary_table[['Average Reading Score','Average Math Score']].applymap(format_round2)
schools_summary_table[['% Passing Reading','% Passing Math', 'Overall Passing Rate']] = schools_summary_table[['% Passing Reading','% Passing Math', 'Overall Passing Rate']].applymap(format_percent)
schools_summary_table[['Total Students']] = schools_summary_table[['Total Students']].applymap(format_round0)